# Personalized Marketing Campagin Content Creation 
## Use case

Assuming you are a marketing campaign manager, you're going to promote the flight ticket for XYZ Airline. At first, AI system will help you find out who are the target users; and then generate marketing promotion template for thoes target users, to protect your brand reputation and mitigate the risk of generative AI, system will use content moderation engine to inspect the content, if the compliance confidence level higher than 99%, system will save the email template into Amazon SES, marketing manager will use it sending the email to target user list. 

There has three AI engines
1. Recommendation engine - Amazon Personalize service
2. Content Generative engine - Amazon Bedrock service
3. Content Moderation engine - Amazon Comprehend service

The AI system pipelein will be - 

Marketing request-->Personalize-->retrive medata-->combine with PromptTemplate--> Amazon Bedrock--> Amazon Comprehend-->send to Amazon SES

## Introduction

In this notebook we show how to generate personalized marketing campaign promotion for email, by contextual metadata and question template. 

LangChain is a framework for developing applications powered by language models. The key aspects of this framework allow us to augment the Large Language Models by chaining together various components to create advanced use cases.

In this notebook we will use the Bedrock API provided by LangChain. The prompt used in this example creates a custom LangChain prompt template for adding context to the text generation request. 


LangChain is a framework for developing applications powered by language models. The key aspects of this framework allow us to augment the Large Models and enable us to perform tasks which meet desired goals and unlock various use-cases.



#### Pre-requisites
Before we get started with the implementation we have to make sure that the required boto3 and botocore packages are installed. These will be used to leverage the Amazon Bedrock API client.

Additionally we would need langchain one of  the latest versions 0.0.190 which has Amazon Bedrock class implemented under llms module. Also we are installing the transformers framework from HuggingFace, which we will use to quickly count the number of tokens in the input prompt.

In [ ]:
%pip install --upgrade pip

In [ ]:
!ls -l

In [ ]:
!python3 -m pip install dependencies/boto3-1.26.162-py3-none-any.whl
!python3 -m pip install dependencies/botocore-1.29.162-py3-none-any.whl

In [ ]:
%pip install langchain==0.0.190 --quiet
%pip install transformers==4.24.0 --quiet

In [ ]:
%pip install ipywidgets

# install timer counter 
%pip install tqdm


In [ ]:
import json
import ipywidgets as widgets
from IPython.display import display

from langchain import PromptTemplate


In [ ]:
import boto3
import json
import os
import sys

boto3_bedrock  = None


## Running Amazon Personalize user segmentation job, it will generate segments of target users

In [ ]:
personalize = boto3.client(service_name = 'personalize')
s3 =boto3.client('s3')

In [ ]:
# Amazon personalize user segmentation batch job, input is target promotion flight ticket, output result is target user group in gold membership

import datetime
import boto3

current_datetime = datetime.datetime.now()
formatted_datetime = current_datetime.strftime("%Y%m%d%H%M")

job_name = f"user-segmentation-{formatted_datetime}"

create_batch_segment_response = personalize.create_batch_segment_job(
    jobName=job_name,
    solutionVersionArn='arn:aws:personalize:us-east-1:<AWS account id>:solution/user-segmentation-v1/8615045e',
    numResults=10,
    jobInput={
        "s3DataSource": {
            "path": "s3://<AWS account id>-us-east-1-personalize-user-segmentation-demo/input/user-segment-request.json"
        }
    },
    jobOutput={
        "s3DataDestination": {
            "path": "s3://<AWS account id>-us-east-1-personalize-user-segmentation-demo/output/"
        }
    },
    roleArn='arn:aws:iam::<AWS account id>:role/service-role/AmazonSageMaker-ExecutionRole-20230616T101619'
)

batch_segment_job_arn = create_batch_segment_response['batchSegmentJobArn']
print(batch_segment_job_arn)


## Prepare Content generative engine, invoke the Bedrock LLM Model

for more details for the parameters please refer to the bedrock api page

In [ ]:
import boto3
import json
bedrock = boto3.client(
 service_name='bedrock',
 region_name='us-east-1',
 endpoint_url='https://bedrock.us-east-1.amazonaws.com'
)

In [ ]:
bedrock.list_foundation_models()

In [ ]:
from langchain.llms.bedrock import Bedrock

# textgen_llm = Bedrock(model_id="amazon.titan-tg1-large", client=boto3_bedrock)

inference_modifier = {'max_tokens_to_sample':4096, 
                      "temperature":0.7,
                      "top_k":250,
                      "top_p":1,
                      "stop_sequences": ["\n\nHuman"]
                     }

textgen_llm = Bedrock(model_id = "anthropic.claude-v1",
                    client = boto3_bedrock, 
                    model_kwargs = inference_modifier 
                    )


## Prepare contextual item medata, and question template 

In [ ]:
def read_json_data(file_path):
    with open(file_path, "r") as json_file:
        data = json.load(json_file)
    return data

def on_confirm_button_click(b):
    selected_metadata_file = metadata_dropdown.value
    selected_template_file = template_dropdown.value

    metadata = read_json_data(selected_metadata_file)
    print("Metadata JSON data:")
    print(json.dumps(metadata, indent=4))

    template = read_json_data(selected_template_file).get("Question", "")
    print("Ticketing Template:")
    print(template)

metadata_dropdown = widgets.Dropdown(
    options=["test-metadata.json", "other-metadata.json"],  # Add more options if needed
    description="Metadata File:"
)

template_dropdown = widgets.Dropdown(
    options=["ticketing-template.json", "other-template.json"],  # Add more options if needed
    description="Template File:"
)

confirm_button = widgets.Button(description="Confirm button")
confirm_button.on_click(on_confirm_button_click)

display(metadata_dropdown, template_dropdown, confirm_button)



## Prepare the prompting template by metadata and question template

In [ ]:
def update_output():
    with open("test-metadata.json", "r") as json_file:
        metadata = json.load(json_file)

    with open("ticketing-template.json", "r") as json_file:
        ticketing_template = json.load(json_file)

    input_variables = list(metadata.keys())
    template = ticketing_template.get("Question", "")

    multi_var_prompt = PromptTemplate(
        input_variables=input_variables,
        template=template
    )

    prompt = multi_var_prompt.format(**metadata)

    output.clear_output()
    with output:
        print(prompt)
    
    
    return prompt    


In [ ]:
# Create the output widget
output = widgets.Output()

# Create the button widget
button = widgets.Button(description="Generate Question")

# Define the event handler for the button
button.on_click(update_output)

# Display the button and output widget
display(button, output)

prompt = update_output()

## Optional, Prompt Chaining
if you have an example template, and you want LLM to learn it, and generate new content. 

https://docs.anthropic.com/claude/docs/prompt-chaining

In [ ]:
#optional, if you have example email template, and you want bedrock to refer it as a baseline example 

import json

# Read the JSON file and extract paragraphs
with open("baseline-sample.json", "r") as json_file:
    data = json.load(json_file)
    title = data.get("Email title", "")
    body = data.get("Email body", "")


# Additional statement
additional_statement = "Here is a good example for you."

# Combine paragraphs with prompt and additional statement
prompt = f"{prompt}\n\n{additional_statement}\n\n{''.join(title)}\n\n{''.join(body)}"

# Print the combined text
print(prompt)


In [ ]:
num_tokens = textgen_llm.get_num_tokens(prompt)
print(f"Our prompt has {num_tokens} tokens")

## Generate the content by the request of prompting question

invoke using the prompt tempalate and expect to see a curated response back

In [ ]:
import time
from tqdm import tqdm

# Assuming you already have the 'textgen_llm' function defined
# prompt = "Your prompt goes here"
response = textgen_llm(prompt)

# Measure the time taken for text generation
start_time = time.time()
email = response[response.index('\n')+1:]
end_time = time.time()
responding_time = end_time - start_time

# Print the responding time
print("Responding time:", responding_time, "seconds")

# Show the progress bar while printing the email
#print("Generating email---->")
#for char in tqdm(email):
#    print(char, end='', flush=True)
#    time.sleep(0.02)  # A small delay to make the progress bar visible
print("\nEmail generation complete.")


In [ ]:
#response = textgen_llm(prompt)

#email = response[response.index('\n')+1:]  

print(email)

## Sending the content into text content moderation engine, and check the complain, if the score is higher than 0.99, it will pass

In [ ]:
# send the email title and body for text content moderation checking

import boto3

region = 'us-east-1'
comprehend = boto3.client('comprehend', region_name=region)

moderation_response = comprehend.classify_document(
    Text=email,
    EndpointArn='arn:aws:comprehend:us-east-1:696784033931:document-classifier-endpoint/demo'
)
print(moderation_response)


## Save the content in JSON file

In [ ]:
import json

# Assuming you already have the 'email' variable with the email content
# If not, you can replace this with the appropriate content.

email_content = email.strip()

# Create a dictionary with the email content
email_dict = {"email": email_content}

# Define the filename for the JSON file
json_filename = "email_content_english.json"

# Save the dictionary as a JSON file
with open(json_filename, "w") as json_file:
    json.dump(email_dict, json_file, indent=4)

print(f"The email content has been saved as {json_filename}.")

In [ ]:


import json
import re

# Step 1: Read the content from the original JSON file
with open('email_content_english.json', 'r', encoding='utf-8') as file:
    data_str = file.read()
    print("Data read from the file:")
    print(data_str)

# Step 2: Extract the email content as string from the nested JSON-like structure
try:
    data = json.loads(data_str)
    email_content_str = data.get('email')
    print("Email content as string:")
    print(email_content_str)

    # Step 3: Clean the email content string from invalid control characters
    email_content_str_cleaned = re.sub(r'[\t\n\r]', '', email_content_str)

    # Step 4: Extract the actual JSON data from the cleaned email_content_str
    email_data_start = email_content_str_cleaned.find("{")
    email_data_end = email_content_str_cleaned.rfind("}") + 1
    email_data_str = email_content_str_cleaned[email_data_start:email_data_end]
    email_content = json.loads(email_data_str)

    # Step 5: Check if the required fields ('Email title' and 'Email body') exist in the loaded JSON data
    if isinstance(email_content, dict) and "Email title" in email_content and "Email body" in email_content:
        # Step 6: Extract the required fields ('Email title' and 'Email body')
        email_title = email_content["Email title"]
        email_body = email_content["Email body"]

        # Step 7: Create a new dictionary with the extracted data
        extracted_data = {
            "Email title": email_title,
            "Email body": email_body
        }

        # Step 8: Write the new JSON object into a new file
        with open('new_email_content.json', 'w', encoding='utf-8') as file:
            json.dump(extracted_data, file, indent=4)

        print("Extraction and saving completed.")
    else:
        print("The required fields ('Email title' and 'Email body') are missing in the JSON data.")
except json.JSONDecodeError as e:
    print("Error decoding JSON data:", e)






## Save the content into Amazon Simple Email service template

In [ ]:

#You can create up to 20,000 email templates in each AWS Region.

import boto3
import json
from datetime import datetime

# Configure AWS credentials (make sure you have appropriate IAM permissions)
aws_access_key_id = "<your id>"
aws_secret_access_key = "<your key>"
aws_region = "us-east-1"

# Load the email template from the JSON file
with open("new_email_content.json", "r") as json_file:
    email_template = json.load(json_file)

# Generate dynamic TemplateName
current_datetime = datetime.now().strftime("%Y%m%d%H%M%S")
template_name = f"campaign{current_datetime}"

# Create a new SES client
ses_client = boto3.client(
    "ses",
    aws_access_key_id=aws_access_key_id,
    aws_secret_access_key=aws_secret_access_key,
    region_name=aws_region,
)

# Create the email template
response = ses_client.create_template(
    Template={
        "TemplateName": template_name,
        "SubjectPart": email_template["Email title"],
        "HtmlPart": email_template["Email body"],
        "TextPart": "For demo email",  # You can provide a text version here if needed
    }
)


In [ ]:
print (template_name)

In [ ]:

# Get the email template
response = ses_client.get_template(TemplateName=template_name)

# Extract the details of the email template
template_subject = response['Template']['SubjectPart']
template_html_body = response['Template']['HtmlPart']
template_text_body = response['Template']['TextPart']

# Display the template details
print(f"Template Name: {template_name}")
print(f"Template Subject: {template_subject}")
print(f"Template HTML Body:\n{template_html_body}")
print(f"Template Text Body:\n{template_text_body}")

## Marketing team can use the template to send email to target users

In [ ]:


# Define the email address to send the email to
recipient_email = "target_user_email"  # Replace with the recipient's email address

# Define replacement data for the template placeholders
replacement_data = {
    "Customer name": "name",
    "phone number": "number",
    "email address": "email",
    "date": "20-08-2023"
}

# Send the email using the email template
response = ses_client.send_templated_email(
    Source="sender email address",  # Replace with the verified sender email address in Amazon SES
    Destination={
        "ToAddresses": [recipient_email],
    },
    Template=template_name,
    TemplateData=json.dumps(replacement_data),
)

print("Email sent successfully.")


# Asia language support

## Bahasa Indonesia

In [ ]:
from langchain import PromptTemplate

# Bahasa Indonesia
# Create a prompt template that has multiple input variables
multi_var_prompt = PromptTemplate(
    input_variables=["DSTCity", "Season", "Airline", "memberClass"], 
    template="""I will promote flight ticket of {DSTCity} during {Season} to my customer, 
    I want to generate an attractive e-mail title and body to promote the flight ticket of Airline {Airline} to the city of {DSTCity} during {Season} for {memberClass} membership, 
    pls help to write a body of words with landscape itinerary details, with an attractive title to help me to promote the flight ticket to the customers. please generate the content for the promotion email in Bahasa Indonesia with an attractive title and 400 words body based on the metadata and prompt template.
   """
    
)

# Pass in values to the input variables
prompt = multi_var_prompt.format(DSTCity="Tokyo", 
                                 Season="April and May", 
                                 Airline="Singapore Airline",
                                 memberClass="Gold"
     )


In [ ]:
num_tokens = textgen_llm.get_num_tokens(prompt)
print(f"Our prompt has {num_tokens} tokens")

In [ ]:
response = textgen_llm(prompt)

email = response[response.index('\n')+1:]  

print(email)

In [ ]:
import json

# Assuming you already have the 'email' variable with the email content
# If not, you can replace this with the appropriate content.

email_content = email.strip()

# Create a dictionary with the email content
email_dict = {"email": email_content}

# Define the filename for the JSON file
json_filename = "email_content_Bahasa.json"

# Save the dictionary as a JSON file
with open(json_filename, "w") as json_file:
    json.dump(email_dict, json_file, indent=4)

print(f"The email content has been saved as {json_filename}.")


## Thai language

In [ ]:
from langchain import PromptTemplate

# Thai language
# Create a prompt template that has multiple input variables
multi_var_prompt = PromptTemplate(
    input_variables=["DSTCity", "Season", "Airline", "memberClass"], 
    template="""I will promote flight ticket of {DSTCity} during {Season} to my customer, 
    I want to generate an attractive e-mail title and body to promote the flight ticket of Airline {Airline} to the city of {DSTCity} during {Season} for {memberClass} membership, 
    pls help to write a body of words with landscape itinerary details, with an attractive title to help me to promote the flight ticket to the customers. please generate the content for the promotion email in Thai language with an attractive title and 400 words body based on the metadata and prompt template.
   """
    
)

# Pass in values to the input variables
prompt = multi_var_prompt.format(DSTCity="Tokyo", 
                                 Season="April and May", 
                                 Airline="Singapore Airline",
                                 memberClass="Gold"
     )


In [ ]:
num_tokens = textgen_llm.get_num_tokens(prompt)
print(f"Our prompt has {num_tokens} tokens")

In [ ]:
response = textgen_llm(prompt)

email = response[response.index('\n')+1:]  

print(email)

## Vietnamese language

In [ ]:
from langchain import PromptTemplate

# Vietnamese language
# Create a prompt template that has multiple input variables
multi_var_prompt = PromptTemplate(
    input_variables=["DSTCity", "Season", "Airline", "memberClass"], 
    template="""I will promote flight ticket of {DSTCity} during {Season} to my customer, 
    I want to generate an attractive e-mail title and body to promote the flight ticket of Airline {Airline} to the city of {DSTCity} during {Season} for {memberClass} membership, 
    pls help to write a body of words with landscape itinerary details, with an attractive title to help me to promote the flight ticket to the customers. please generate the content for the promotion email in Vietnamese language with an attractive title and 400 words body based on the metadata and prompt template.
   """
    
)

# Pass in values to the input variables
prompt = multi_var_prompt.format(DSTCity="Tokyo", 
                                 Season="April and May", 
                                 Airline="Singapore Airline",
                                 memberClass="Gold"
     )


In [ ]:
num_tokens = textgen_llm.get_num_tokens(prompt)
print(f"Our prompt has {num_tokens} tokens")

In [ ]:
response = textgen_llm(prompt)

email = response[response.index('\n')+1:]  

print(email)

## Chinese language

In [ ]:
from langchain import PromptTemplate

# Vietnamese language
# Create a prompt template that has multiple input variables
multi_var_prompt = PromptTemplate(
    input_variables=["DSTCity", "Season", "Airline", "memberClass"], 
    template="""I will promote flight ticket of {DSTCity} during {Season} to my customer, 
    I want to generate an attractive e-mail title and body to promote the flight ticket of Airline {Airline} to the city of {DSTCity} during {Season} for {memberClass} membership, 
    pls help to write a body of words with landscape itinerary details, with an attractive title to help me to promote the flight ticket to the customers. please generate the content for the promotion email in chinese language with an attractive title and 400 words body based on the metadata and prompt template.
   """
    
)

# Pass in values to the input variables
prompt = multi_var_prompt.format(DSTCity="Tokyo", 
                                 Season="April and May", 
                                 Airline="Singapore Airline",
                                 memberClass="Gold"
     )

In [ ]:
num_tokens = textgen_llm.get_num_tokens(prompt)
print(f"Our prompt has {num_tokens} tokens")

In [ ]:
response = textgen_llm(prompt)

email = response[response.index('\n')+1:]  

print(email)

#### Summary

To conclude we learnt that invoking the LLM without any context might not yeild the desired results. By adding contextual data and further using the the prompt template to constrain the output from the LLM we are able to successfully get our desired output

## Text to Image (optional)

In [ ]:
%pip install --quiet "pillow>=9.5,<10"

In [ ]:
import base64
import io
import json
import os
import sys

In [ ]:
import boto3
from PIL import Image
bedrock_client = boto3.client('bedrock' , 'us-east-1', endpoint_url='https://bedrock.us-east-1.amazonaws.com')
bedrock_client.list_foundation_models()

## Prepare prompting
The promopting can be generated by last step of text generation

In [ ]:
prompt = "I will promote flight ticket of Airline ticket, from Singapore to Tokyo, during ['April', 'May'] for Gold membership, pls create a cover photographic, without human"

In [ ]:

negative_prompts = [
    "poorly rendered",
    "poor background details",
    "Not realistic enough",
    "poorly drawn people",
    "poorly drawn human eyes",
    "poorly drawn human nose",
    "poorly drawn human hair",
    "poorly drawn human finger",
    "missing human fingers",
    "poorly drawn aircraft",
    "poorly quality of font",
    "pixels too lower",
    "disfigured people features",
]
style_preset = "photographic"  # (e.g. photographic, digital-art, cinematic, ...)
#prompt = "photo taken from above of an italian landscape. cloud is clear with few clouds. Green hills and few villages, a lake"

### The Stability.ai Diffusion models support the following controls. 

cfg_scale: Prompt strength– Determines how much the final image portrays the prompt random generations. The range is 0—30, and the default value is 10.
the "cfg_scale" essentially governs how much the image looks closer to the prompt or input image. The higher the CFG scale, the more the image will match your prompt. Conversely, a lower CFG scale value produces a better-quality image that may differ from the original prompt or image

In Stable Diffusion, CFG stands for Classifier Free Guidance scale. CFG is the setting that controls how closely Stable Diffusion should follow your text prompt. It is applied in text-to-image (txt2img) and image-to-image (img2img) generations.

The higher the CFG value, the more strictly it will follow your prompt, in theory. The default value is 10, which gives a good balance between creative freedom and following your direction. A value of 1 will give Stable Diffusion almost complete freedom, whereas values above 15 are quite restrictive.

step: Generation step determines how many times the image is sampled. More steps can result in a more accurate result. The range is 0—150, and the default value is 5.

Seed: The seed determines the initial noise setting. If you use the same seed and the same settings as a previous run, inference creates a similar image. The seed value is a random number.


style_preset: the parameter includes enhance, anime, photographic, digital-art, comic-book, fantasy-art, line-art, analog-film, neon-punk, isometric, low-poly, origami, modeling-compound, cinematic, 3d-model, pixel-art, and tile-texture. This list of style presets is subject to change; refer to the latest release and documentation for updates.

https://platform.stability.ai/docs/api-reference#tag/v1generation/operation/textToImage

In [ ]:
request = json.dumps({
    "text_prompts": (
        [{"text": prompt, "weight": 1.0}]
        + [{"text": negprompt, "weight": -1.0} for negprompt in negative_prompts]
    ),
    "cfg_scale": 10,
    "seed": 11789,
    "steps": 150,
    "style_preset": style_preset,
})
modelId = "stability.stable-diffusion-xl"

response = bedrock_client.invoke_model(body=request, modelId=modelId)
response_body = json.loads(response.get("body").read())

print(response_body["result"])
base_64_img_str = response_body["artifacts"][0].get("base64")
print(f"{base_64_img_str[0:80]}...")

### By decoding our Base64 string to binary, and loading it with an image processing library like Pillow that can read PNG files, we can display and manipulate the image here in the notebook:

In [ ]:
os.makedirs("data", exist_ok=True)
image_1 = Image.open(io.BytesIO(base64.decodebytes(bytes(base_64_img_str, "utf-8"))))
image_1.save("data/image_1.png")
image_1

## Image to Image

Generating images from text is powerful, but in some cases could need many rounds of prompt refinement to get an image "just right".

Rather than starting from scratch with text each time, image-to-image generation lets us modify an existing image to make the specific changes we'd like.

We'll have to pass our initial image in to the API in base64 encoding, so first let's prepare that. You can use either the initial image from the previous section, or a different one if you'd prefer:

In [ ]:
def image_to_base64(img) -> str:
    """Convert a PIL Image or local image file path to a base64 string for Amazon Bedrock"""
    if isinstance(img, str):
        if os.path.isfile(img):
            print(f"Reading image from file: {img}")
            with open(img, "rb") as f:
                return base64.b64encode(f.read()).decode("utf-8")
        else:
            raise FileNotFoundError(f"File {img} does not exist")
    elif isinstance(img, Image.Image):
        print("Converting PIL Image to base64 string")
        buffer = io.BytesIO()
        img.save(buffer, format="PNG")
        return base64.b64encode(buffer.getvalue()).decode("utf-8")
    else:
        raise ValueError(f"Expected str (filename) or PIL Image. Got {type(img)}")




In [ ]:
# Define image_to_image1 as a PIL Image object with your actual image file path)
image_to_image1 = Image.open('image_to_image1.png')

init_image_b64 = image_to_base64(image_to_image1)
print(init_image_b64[:80] + "...")

In [ ]:
#os.makedirs("data", exist_ok=True)
#image_to_image1 = Image.open(io.BytesIO(base64.decodebytes(bytes(init_image_b64, "utf-8"))))
#image_to_image1.save("data/image_to_image1.png")
image_to_image1

In [ ]:
request = json.dumps({
    "text_prompts": (
        [{"text": prompt, "weight": 1.0}]
        + [{"text": negprompt, "weight": -1.0} for negprompt in negative_prompts]
    ),
    "cfg_scale": 10,
    "init_image": init_image_b64,
    "seed": 3661,
    "start_schedule": 0.6,
    "steps": 150,
    "style_preset": style_preset,
})
modelId = "stability.stable-diffusion-xl"

response = bedrock_client.invoke_model(body=request, modelId=modelId)
response_body = json.loads(response.get("body").read())

print(response_body["result"])
image_2_b64_str = response_body["artifacts"][0].get("base64")
print(f"{image_2_b64_str[0:80]}...")

In [ ]:
image_2 = Image.open(io.BytesIO(base64.decodebytes(bytes(image_2_b64_str, "utf-8"))))
image_2.save("data/image_2.png")
image_2